In [ ]:
ls ../data

In [4]:
import sys
import os
sys.path.append('../code')
from io import StringIO
from subprocess import Popen, PIPE, run
from HDR_run import HDR_master

home = '/Users/mahtin'
path = '/Users/martinscience/Dropbox/Icke/Work/somVar/HDRtest'
bam_file = f'{path}/data/002.bam '
chrom = 'chr1'
# chrom = ''
pileup_file = f'{path}/data/001.chr1.pileup'
threads = 8
mut_file = f'{path}/data/002.csv'
genome_split_path = os.path.join(home,'Dropbox/Icke/Work/static/genome/gatk/hg38/split')
HDR_config = {
    "minAltSum": 2,
    "minAltRatio": 0.1,
    "maxAltRatio": 0.85,
    "MINQ": 15,
    "MINSIM": .50,
    "PAD": 100,
    "MINq": 10,
    "MinAltSupport": 5,
    "MinHDRCount": 1,
    "pile2hotspot": '../shell/pile2hotspot.mawk',#
    "pile2hotspot_chrom": '../shell/pile2hotspot_chrom.mawk',
    "editbam": '../shell/editbam.mawk',
    "bam2csv": '../shell/bam2csv.mawk',
    "genome_split_path": genome_split_path # the path to the folder with chrom-split genomes (chr1.fa..)
}
HDR_master(mut_file, bam_file=bam_file, pileup_file=pileup_file, chrom=chrom, threads=threads, HDR_config=HDR_config)

Importing /Users/martinscience/Dropbox/Icke/Work/somVar/HDRtest/data/002.csv for HDR detection
17:07:06 : Starting HDR analysis of /Users/martinscience/Dropbox/Icke/Work/somVar/HDRtest/data/002.csv on chr1 using 8 cores. [MIN_SIM=0.5, PAD=100]
Process 43215 : $ cat /Users/martinscience/Dropbox/Icke/Work/somVar/HDRtest/data/001.chr1.pileup | ../shell/pile2hotspot_chrom.mawk chr1 2
17:07:06 : Detected 4 putative HDR lanes in /Users/martinscience/Dropbox/Icke/Work/somVar/HDRtest/data/001.chr1.pileup.
17:07:06 : Using 8 cores for filtering HDR-rich mutations
17:07:07 : Found no HDR-rich mutations in /Users/martinscience/Dropbox/Icke/Work/somVar/HDRtest/data/001.chr1.pileup


,Chr,Start,End,Ref,Alt,Gene,HDRcand,HDRcount,HDRinfo
0,chr1,3632268,3632268,G,C,WRAP73,0,0,no HDR in vincinity
1,chr1,4712657,4712657,G,A,AJAP1,0,0,no HDR in vincinity
2,chr1,23417366,23417366,T,C,TCEA3,0,0,no HDR in vincinity
3,chr1,39511892,39511892,G,A,BMP8A,0,0,no HDR in vincinity
4,chr1,39769776,39769776,C,T,OXCT2,0,0,no HDR in vincinity
...,...,...,...,...,...,...,...,...,...
172,chr22,50030478,50030478,C,T,TTLL8,0,0,no HDR in vincinity
173,chr22,50030504,50030504,G,T,TTLL8,0,0,no HDR in vincinity
174,chrX,71122601,71122601,A,C,MED12,0,0,no HDR in vincinity
175,chrX,108183752,108183752,-,CTT,COL4A6,0,0,no HDR in vincinity


In [ ]:
ls data

In [ ]:

home = '/Users/mahtin'
path = '/Users/mahtin/Dropbox/Icke/Work/somVar/HDRtest'
bam_file = f'{path}/data/002.bam '
chrom = 'chr1'
# chrom = ''
pileup_file = f'{path}/data/002.pileup'
threads = 8
mut_file = f'{path}/data/002.csv'
mut_df = pd.read_csv(mut_file, sep='\t').loc[:, [
        'Chr', 'Start', 'End', 'Ref', 'Alt', 'Gene']]
# make Chr column categorical for sorting .. and sort
chrom_list = [f"chr{i}" for i in range(23)] + ['chrX', 'chrY']
mut_df['Chr'] = pd.Categorical(mut_df['Chr'], chrom_list)
mut_df = mut_df.sort_values(['Chr', 'Start'])
mut_df = mut_df.query('Chr == @chrom')
mut_df
mut_split = np.array_split(mut_df, 4)

In [ ]:
mut_df = mut_split[0]

In [ ]:
anno_df = mut_df.sort_values(['Chr', 'Start']).iloc[0:,:5]
anno_df

In [ ]:
def reduce_regions(df, padding):
    '''
    takes a mutation list and returns a region list using padding
    overlapping regions are reduced to one using the gap strategy
    '''

    df = df.sort_values('Start')
    df['Start'] = df['Start'] - padding
    df['End'] = df['End'] + padding
    # find the break points
    # if Start is greater than previous End (using shift), this is a gap --> df['gap'] = 1
    df['gap'] = df['Start'].gt(df['End'].shift()).astype('int')
    # id different reads according to gap
    # cumulative sum does not increase at df['gap'] == 0 and so these consecutive stretches are grouped together
    df['gap'] = df['gap'].cumsum()
    # groupby the coverage break group and condense individual coverage islands
    # agg has to contain the neccessary shared columns TransLength because it is needed for coverage computation
    df = df.groupby('gap').agg({'Chr':'first','Start': 'min', 'End':'max'})
    return df.reset_index('gap').drop(columns='gap')

In [ ]:
df = anno_df
anno_df
padding = 100
df['Start'] = df['Start'] - padding
df['End'] = df['End'] + padding
df['gap'] = df['Start'].gt(df['End'].shift()).astype('int')
df['gap'] = df['gap'].cumsum()
df = df.groupby('gap').agg({'Chr':'first','Start': 'min', 'End':'max'}).reset_index().drop(columns='gap')
df

In [ ]:
bed_df = reduce_regions(anno_df, 100)
bed_df

In [ ]:
pd.DataFrame(columns=[
        'Chr',
        'Start',
        'End',
        'Ref',
        'Alt',
        'Gene',
        'HDRcand',
        'HDRcount',
        'HDRinfo',
    ])